In [7]:
## Run when initialise the code
from Code.general_functions import *
import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io

from pandas import DataFrame

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Point, LineString, MultiPolygon
import math
import warnings
import pandas as pd

from tqdm import tqdm
import ast
warnings.filterwarnings(action='ignore')
from momepy import extend_lines
import os
pjr_loc = os.path.dirname(os.getcwd())
from itertools import combinations
import numpy as np
from math import log2

In [8]:

# Define the data folder path
# Construct the folder path for the data using the current working directory
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'places', 'run_SB', 'Santa_barbara_California')

# Load the OSM data into a GeoDataFrame, specifying the 'edges' layer
my_gdf = gpd.read_file(os.path.join(data_folder, 'osm_data.gpkg'), layer='edges')

# Filter out unwanted road types (e.g., motorway, trunk, and their links)
to_remove = my_gdf[~my_gdf['highway'].isin(['motorway', 'trunk', 'motorway_link', 'trunk_link'])]

# Exclude rows where 'name' is NaN or an empty string, then convert CRS to project_crs
df_pro = to_remove[~(to_remove['name'].isna() | (to_remove['name'] == ''))].to_crs(project_crs)

# Function to safely convert valid string representations of lists to actual lists
def convert_to_list(s):
    """
    Converts a string representation of a list into a Python list.
    
    Parameters:
    - s (str): The string to convert.
    
    Returns:
    - The first element of the list if the string is a valid list representation.
    - The original value if conversion fails.
    """
    try:
        return ast.literal_eval(s)[0]  # Extract the first element if the string is a valid list
    except (ValueError, SyntaxError, TypeError):
        return s  # Return the original value if the conversion fails

# Apply the conversion function to 'name' and 'highway' columns
df_pro['name'] = df_pro['name'].apply(convert_to_list)
df_pro['highway'] = df_pro['highway'].apply(convert_to_list)


In [10]:
# vars
str_name = 'name_left'
con_str_name = 'name_right'
# 1.	Copy network
df_analysis = df_pro.copy()

In [11]:
s_join_analysis = gpd.sjoin(df_analysis,df_pro)

In [12]:
s_join_analysis2 = s_join_analysis[s_join_analysis[str_name]!=s_join_analysis[con_str_name]]

In [13]:
group_name = s_join_analysis2.groupby(str_name)

In [14]:
dic_data={}

In [15]:
def connected_streets(str_lines):
    """
    Populate the dictionary with the name of the current street and all its connected streets.
    :param str_lines:
    :return:
    """

    dic_data[str_lines[str_name].iloc[0]] =list(str_lines[con_str_name].unique())

group_name.apply(connected_streets)
dic_data

KeyError: 'is_simplif_left'